In [31]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_squared_error

In [32]:
games = pd.read_csv(f'data/train_data.csv')
games = games[games['completed'] == True]

In [40]:

def backtest(model, games, x_features, label):
    num_games_increment = 10
    errors = []
    min_num_games = games['num_games_into_season'].min()
    max_num_games = games['num_games_into_season'].max()

    for num_games in range(min_num_games, max_num_games, num_games_increment):
        print(num_games)
        train = games[games['num_games_into_season'] < num_games]
        if len(train) == 0:
            continue
        test = games[(num_games + num_games_increment > games['num_games_into_season']) & (games['num_games_into_season'] >= num_games)]
        X = train[x_features]
        y = train[label]
        model.fit(X, y)
        X_test = test[x_features]
        y_test = test[label]
        y_pred = model.predict(X_test)
        test.loc[:,'pred_margin'] = y_pred
        test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2
        errors += list(test['mse'])

        print('Num games: ', num_games, 'MSE on sample: ', round(mean_squared_error(y_test, y_pred), 2), 'MSE on all games: ', round(np.mean(errors), 2))
    return errors

params = {'max_depth': 5, 'learning_rate': 0.014567791445364069, 'n_estimators': 655, 'min_child_weight': 2, 'gamma': 0.11052774751544212, 'subsample': 0.9899289938848144, 'colsample_bytree': 0.9249071617042357, 'reg_alpha': 0.4468005337539522, 'reg_lambda': 0.24513091931966713, 'random_state': 996}
model = XGBRegressor(**params)

x_features = ['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']
label = 'margin'
errors = backtest(model, games, x_features, label)







0
10


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  10 MSE on sample:  247.64 MSE on all games:  247.64
20


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  20 MSE on sample:  245.4 MSE on all games:  246.64
30


...

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  1300 MSE on sample:  227.2 MSE on all games:  170.72
1310


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


Num games:  1310 MSE on sample:  213.11 MSE on all games:  170.82
1320
Num games:  1320 MSE on sample:  236.8 MSE on all games:  170.83


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'pred_margin'] = y_pred
/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1731616626.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:,'mse'] = (test['margin'] - test['pred_margin'])**2


In [46]:
X_train, X_test, y_train, y_test = train_test_split(games[['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'last_year_team_rating*num_games_into_season', 'last_year_opponent_rating*num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']], games['margin'], test_size=0.2, random_state=42)

In [47]:
# use optuna to tune the parameters
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [48]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=100)

[I 2023-01-31 23:19:19,689] A new study created in memory with name: regression
[I 2023-01-31 23:19:27,115] Trial 0 finished with value: 20.50425487433687 and parameters: {'max_depth': 9, 'learning_rate': 0.8899898768032132, 'n_estimators': 742, 'min_child_weight': 8, 'gamma': 0.24483365891808206, 'subsample': 0.683549360316754, 'colsample_bytree': 0.1800994705731504, 'reg_alpha': 0.12299255906397302, 'reg_lambda': 0.06124102918940829, 'random_state': 908}. Best is trial 0 with value: 20.50425487433687.
[I 2023-01-31 23:19:28,169] Trial 1 finished with value: 15.595338601246185 and parameters: {'max_depth': 2, 'learning_rate': 0.6731381641114502, 'n_estimators': 343, 'min_child_weight': 5, 'gamma': 0.20895874738527057, 'subsample': 0.16966601811503212, 'colsample_bytree': 0.6273308127180829, 'reg_alpha': 0.8762323047959243, 'reg_lambda': 0.5423191738212015, 'random_state': 938}. Best is trial 1 with value: 15.595338601246185.
[I 2023-01-31 23:19:30,956] Trial 2 finished with value: 120

KeyboardInterrupt: 

In [ ]:
print('Best parameters', study.best_params)
print('Best value', study.best_value)

In [3]:
games['win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(games[['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'last_year_team_rating*num_games_into_season', 'last_year_opponent_rating*num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']], games['win'], test_size=0.2, random_state=41)

In [60]:
# use optuna to tune the parameters for a classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    return log_loss(y_test, y_pred)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='win_probability')
study.optimize(objective, n_trials=100)

In [32]:
y_pred = [0.5] * len(y_test)
print('Baseline log loss', log_loss(y_test, y_pred))

Baseline log loss 0.6931471805599452


In [9]:
games = pd.read_csv(f'data/train_data.csv')
games = games[games['completed'] == True]
games = games[games['year'] == 2023]
games['win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)
# test the model on the 2023 season
params = {'max_depth': 1, 'learning_rate': 0.6642774658597874, 'n_estimators': 365, 'min_child_weight': 3, 'gamma': 0.9138018877462927, 'subsample': 0.9850665433158399, 'colsample_bytree': 0.6714438416337997, 'reg_alpha': 0.6338885144969151, 'reg_lambda': 0.18021132749565166, 'random_state': 23}
model = XGBClassifier(**params)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
print('Log loss', log_loss(y_test, y_pred))

NameError: name 'X_train' is not defined

In [39]:

from sklearn.metrics import log_loss

def backtest(model, games, x_features, label):
    num_games_increment = 10
    errors = []
    min_num_games_into_season = games['num_games_into_season'].min()
    max_num_games_into_season = games['num_games_into_season'].max()
    for num_games in range(min_num_games_into_season, max_num_games_into_season, num_games_increment):
        train = games[games['num_games_into_season'] < num_games]
        if len(train) == 0:
            continue
        test = games[(games['num_games_into_season'] >= num_games) & (games['num_games_into_season'] < num_games + num_games_increment)]
        X = train[x_features]
        y = train[label]
        model.fit(X, y)
        X_test = test[x_features]
        y_test = test[label]
        y_pred = model.predict_proba(X_test)[::,1]
        # get log loss
        test['pred_prob'] = y_pred
        score = log_loss(test['team_win'], test['pred_prob'], labels=[0, 1])
        errors += [score] * len(test)

        print('Num games: ', num_games, 'log loss on sample: ', score, 'log loss overall: ', np.mean(errors))
    return errors

games['team_win'] = games.apply(lambda row: 1 if row['margin'] > 0 else 0, axis=1)
params = {'max_depth': 5, 'learning_rate': 0.01337501236333186, 'n_estimators': 615, 'min_child_weight': 6, 'gamma': 0.22171810700204012, 'subsample': 0.23183800840898533, 'colsample_bytree': 0.29826505641378537, 'reg_alpha': 0.5869931848470185, 'reg_lambda': 0.01392437600344064, 'random_state': 931}
model = XGBClassifier(**params)

x_features = ['team_rating', 'opponent_rating', 'team_win_total_future', 'opponent_win_total_future', 'last_year_team_rating', 'last_year_opponent_rating', 'num_games_into_season', 'team_last_10_rating', 'opponent_last_10_rating', 'team_last_5_rating', 'opponent_last_5_rating', 'team_last_3_rating', 'opponent_last_3_rating', 'team_last_1_rating', 'opponent_last_1_rating', 'team_days_since_most_recent_game', 'opponent_days_since_most_recent_game']
label = 'team_win'
errors = backtest(model, games, x_features, label)

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  10 log loss on sample:  0.7040308767351611 log loss overall:  0.7040308767351612


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  20 log loss on sample:  0.6183433779793926 log loss overall:  0.665765848664837


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


...

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  1300 log loss on sample:  0.618563739921559 log loss overall:  0.6219002507969436


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred


Num games:  1310 log loss on sample:  0.7351226644856589 log loss overall:  0.6221512505739327
Num games:  1320 log loss on sample:  0.6815819342931112 log loss overall:  0.6221625413124013


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_10067/1685079216.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_prob'] = y_pred
